<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="300" alt="cognitiveclass.ai logo"  />
</center>

# **Forecasting of Breast Cancer on medical measurement**
# Lab 2. Data Wrangling

Estimated time needed: **30** minutes

## Objectives

After completing this lab you will be able to:

*   Handle missing values
*   Correct data format
*   Standardize and normalize data


<h2>Table of Contents</h2>

<div class="alert alert-block alert-info" style="margin-top: 20px">
<ul>
    <li><a href="https://#identify_handle_missing_values">Identify and handle missing values</a>
        <ul>
            <li><a href="https://#identify_missing_values">Identify missing values</a></li>
            <li><a href="https://#deal_missing_values">Deal with missing values</a></li>
            <li><a href="https://#correct_data_format">Correct data format</a></li>
        </ul>
    </li>
    <li><a href="https://#data_standardization">Data standardization</a></li>
    <li><a href="https://#data_normalization">Data normalization (centering/scaling)</a></li>
    <li><a href="https://#binning">Binning</a></li>
    <li><a href="https://#indicator">Indicator variable</a></li>
</ul>

</div>

<hr>


<h2>What is the purpose of data wrangling?</h2>


Data wrangling is the process of converting data from the initial format to a format that may be better for analysis.


<h3>How to transform months to years?</h3>


<h3>Import data</h3>
<p>
You can find the "Breast Cancer Dataset" from the following link: <a href="https://www.kaggle.com/datasets/gunesevitan/breast-cancer-metabric">https://www.kaggle.com/datasets/gunesevitan/breast-cancer-metabric</a>.
We will be using this dataset throughout this course.
</p>


<h4>Import pandas</h4>

Now, let's import libraries that we will use

In [ ]:
conda install -c anaconda scikit-learn

In [ ]:
import pandas as pd
import matplotlib.pylab as plt
import numpy as np

from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import confusion_matrix, recall_score
from sklearn import preprocessing

%matplotlib inline
import matplotlib as plt
from matplotlib import pyplot

<h2>Reading the dataset from the URL and adding the related headers</h2>


First, we assign the URL of the dataset to "filename".


In [ ]:
filename = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-GPXX0XQ4EN/breast_cancer.csv"

Use the Pandas method <b>read_csv()</b> to load the data from the web address.


In [ ]:
df = pd.read_csv(filename)

Let's read the data in df2 and drop all the rows that have no data

In [ ]:
df2=df.dropna().reset_index(drop=True)
df2.head(10)

Now let's write in the dictionary <b>col</b> the columns in which we will generate empty fields.

In [ ]:
cols = {'Age at Diagnosis': df2[['Age at Diagnosis']], 'Primary Tumor Laterality': df2[['Primary Tumor Laterality']],
        'Nottingham prognostic index': df2[['Nottingham prognostic index']], 'Overall Survival (Months)': df2[['Overall Survival (Months)']], "Patient's Vital Status": df2[["Patient's Vital Status"]]}

col = {'Age at Diagnosis': 0.07, 'Primary Tumor Laterality': 0.1, 'Nottingham prognostic index': 0.05, 'Overall Survival (Months)': 0.03, "Patient's Vital Status": 0.01}
for c in col:
    m = np.random.rand(len(df2))
    mask = m < col[c]
    df2.loc[mask, c] = np.NaN


Use the method <b>head()</b> to display the first five rows of the dataframe.


In [ ]:
# To see what the data set looks like, we'll use the head() method.
df2.head()

As we can see, several NaN appeared in the dataframe; those are missing values which may hinder our further analysis.

<div>So, how do we identify all those missing values and deal with them?</div> 

<b>How to work with missing data?</b>

Steps for working with missing data:

<ol>
    <li>Identify missing data</li>
    <li>Deal with missing data</li>
    <li>Correct data format</li>
</ol>


<h2 id="identify_handle_missing_values">Identify and handle missing values</h2>

<h3 id="identify_missing_values">Identify missing values</h3>


<h4>Evaluating for Missing Data</h4>

The missing values are converted by default. We use the following functions to identify these missing values. There are two methods to detect missing data:

<ol>
    <li><b>.isnull()</b></li>
    <li><b>.notnull()</b></li>
</ol>
The output is a boolean value indicating whether the value that is passed into the argument is in fact missing data.


In [ ]:
missing_data = df2.isnull()
missing_data.head(5)

"True" means the value is a missing value while "False" means the value is not a missing value.


<h4>Count missing values in each column</h4>
<p>
Using a for loop in Python, we can quickly figure out the number of missing values in each column. As mentioned above, "True" represents a missing value and "False" means the value is present in the dataset.  In the body of the for loop the method ".value_counts()" counts the number of "True" values. 
</p>


In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")

Based on the summary above, each column contains more than 1000 rows of data and six of the columns containing missing data:

<ol>
    <li>"Age at Diagnosis"</li>
    <li>"Primary Tumor Laterality"</li>
    <li>"Nottingham prognostic index"</li>
    <li>"Overall Survival (Months)"</li>
    <li>"Patient's Vital Status"</li>
</ol>


<h3 id="deal_missing_values">Deal with missing data</h3>
<b>How to deal with missing data?</b>

<ol>
    <li>Drop data<br>
        a. Drop the whole row<br>
        b. Drop the whole column
    </li>
    <li>Replace data<br>
        a. Replace it by mean<br>
        b. Replace it by frequency<br>
        c. Replace it based on other functions
    </li>
</ol>


Whole columns should be dropped only if most entries in the column are empty. In our dataset, none of the columns are empty enough to drop entirely.
We have some freedom in choosing which method to replace data; however, some methods may seem more reasonable than others. We will apply each method to many different columns:

<b>Replace by mean:</b>

<ul>
    <li>"Age at Diagnosis": replace them with mean</li>
    <li>"Nottingham prognostic index": replace them with mean</li>
    <li>"Overall Survival (Months)": replace them with mean</li>
</ul>

<b>Replace by frequency:</b>

<ul>
    <li>"Primary Tumor Laterality": replace them with "Left".
        <ul>
            <li>Reason: More than half of the data is "Left". Since "Left" is the most common, this is the most likely</li>
        </ul>
    </li>
</ul>

<b>Drop the whole row:</b>

<ul>
    <li>"Patient's Vital Status": simply delete the whole row
    </li>
</ul>


<h4>Calculate the mean value for the "Age at Diagnosis" column </h4>


In [ ]:
avg_age_loss = df2["Age at Diagnosis"].astype("float").mean(axis=0)
print("Age at Diagnosis:", round(avg_age_loss, 2))

<h4>Replace "NaN" with mean value in "Age at Diagnosis" column</h4>


In [ ]:
df2["Age at Diagnosis"].replace(np.nan, avg_age_loss, inplace=True)

<h4>Let's calculate the error between the replaced data and the original data</h4>

In [ ]:
mape_age = mean_absolute_percentage_error(cols['Age at Diagnosis'], df2['Age at Diagnosis'])
print('Error of Age at Diagnosis: ', round(mape_age, 2) * 100, '%')

<h4>Calculate the mean value for the "Nottingham prognostic index" column</h4>


In [ ]:
avg_nottingham=df2['Nottingham prognostic index'].astype('float').mean(axis=0)
print("Nottingham prognostic index:", round(avg_nottingham, 2))

<h4>Replace "NaN" with the mean value in the "Nottingham prognostic index" column</h4>


In [ ]:
df2["Nottingham prognostic index"].replace(np.nan, avg_nottingham, inplace=True)

<h4>Let's calculate the error between the replaced data and the original data</h4>

In [ ]:
mape_nottingham = mean_absolute_percentage_error(cols['Nottingham prognostic index'], df2['Nottingham prognostic index'])
print('Error of Nottingham prognostic index: ', round(mape_nottingham, 2) * 100, '%')

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #1: </h1>

<b>Based on the example above, replace NaN in "Overall Survival (Months)" column with the mean value and calculate the error.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
#Calculate the mean value for "stroke" column
avg_survival = df2['Overall Survival (Months)'].astype('float').mean(axis=0)
print("Overall Survival (Months):", avg_survival)

# replace NaN by mean value in "stroke" column
df2['Overall Survival (Months)'].replace(np.nan, avg_survival, inplace=True)

#Let's calculate the error between the replaced data and the original data
mape_survival = mean_absolute_percentage_error(cols['Overall Survival (Months)'], df2['Overall Survival (Months)'])
print('Error of Overall Survival (Months): ', round(mape_survival, 2) * 100, '%')
```

</details>


To see which values are present in a particular column, we can use the ".value_counts()" method:


In [ ]:
df2['Primary Tumor Laterality'].value_counts()

We can see that "Left" is the most common type. We can also use the ".idxmax()" method to calculate the most common type automatically:


In [ ]:
df2['Primary Tumor Laterality'].value_counts().idxmax()

The replacement procedure is very similar to what we have seen previously:


In [ ]:
#replace the missing 'Primary Tumor Laterality' values by the most frequent
df2["Primary Tumor Laterality"].replace(np.nan, "Left", inplace=True)

<h4>Let's calculate the error between the replaced data and the original data</h4>

In [ ]:
confusion_matrix(cols['Primary Tumor Laterality'], df2[['Primary Tumor Laterality']])

mape_primary = recall_score(cols['Primary Tumor Laterality'], df2[['Primary Tumor Laterality']], average='macro')
print('Error of Primary Tumor Laterality: ', round(1 - mape_primary, 2) * 100, '%')

Finally, let's drop all rows that do not have Patient's Vital Status data:


In [ ]:
# simply drop whole row with NaN in "Patient's Vital Status" column
df2.dropna(subset=["Patient's Vital Status"], axis=0, inplace=True)

# reset index, because we droped two rows
df2.reset_index(drop=True, inplace=True)

In [ ]:
df2.head()

We can notice that the errors we calculated are insignificant which is acceptable. So we can go back to our original <b>df</b> data and replace the empty values.

Let's create two arrays in which we will write the names of the columns in which we will replace empty values. Data from the first array will be replaced by the mean. Data from the second array are replaced by the most frequent.


In [ ]:
col_mean = ["Age at Diagnosis", "Nottingham prognostic index", "Overall Survival (Months)",
            "Relapse Free Status (Months)", "Tumor Size", ]
col_freq = ["Type of Breast Surgery", "Cellularity", "Chemotherapy", "Pam50 + Claudin-low subtype", "Cohort",
            "ER status measured by IHC", "ER Status", "Neoplasm Histologic Grade", "HER2 status measured by SNP6", "HER2 Status", "Tumor Other Histologic Subtype", "Hormone Therapy", "Inferred Menopausal State", "Integrative Cluster", "Primary Tumor Laterality", "Lymph nodes examined positive", "Mutation Count", "Oncotree Code", "Overall Survival Status", "PR Status", "Radio Therapy", "Relapse Free Status", "3-Gene classifier subtype", "Tumor Stage", "Patient's Vital Status"]

Calculate the mean values for the <b>col_mean</b> array and replace them.

In [ ]:
#Calculate the mean values for "col_mean" array

for c in col_mean:
    avg_c = df[c].astype('float').mean(axis=0)
    print(c, ":", round(avg_c, 2))
    df[c].replace(np.nan, avg_c, inplace=True)


Let's determine the most common that value in the array using the ".idxmax()" and replace them.

In [ ]:
#replace the missing values for "col_freq" array by the most frequent

for c in col_freq:
    max_value = df[c].value_counts().idxmax()
    #replace the missing values by the most frequent
    df[c].replace(np.nan, max_value, inplace=True)

df.head()

<b>Good!</b> Now, we have a dataset with no missing values.


<h3 id="correct_data_format">Correct data format</h3>
<b>We are almost there!</b>
<p>The last step in data cleaning is checking and making sure that all data is in the correct format (int, float, text or other).</p>

In Pandas, we use:

<p><b>.dtype()</b> to check the data type</p>
<p><b>.astype()</b> to change the data type</p>


<h4>Let's list the data types for each column</h4>


In [ ]:
df.dtypes

<p>As we can see above, some columns are not of the correct data type. Numerical variables should have type 'float' or 'int', and variables with strings such as categories should have type 'object'. For example, 'Cohort' and 'Neoplasm Histologic Grade' they are shown as type 'float', although they must be of type 'int'. We have to convert data types into a proper format for each column using the "astype()" method.</p>


<h4>Convert data types to proper format</h4>


In [ ]:
col_int = ["Cohort", "Neoplasm Histologic Grade","Lymph nodes examined positive", "Mutation Count", "Tumor Stage"]

col_cat = list(df.select_dtypes(include=['object']).columns)
#exceprt column "Patient ID"
col_cat = col_cat[1:]

col_bool = ["Chemotherapy", "Hormone Therapy", "Radio Therapy"]

df[col_int] = df[col_int].astype("int")
df[col_cat] = df[col_cat].astype("category")
for c in col_bool:
    df[c] = df[c].map({'Yes': True, 'No': False})

df[col_bool] = df[col_bool].astype("bool")

<h4>Let us list the columns after the conversion</h4>


In [ ]:
df.dtypes

<b>Wonderful!</b>

Now we have finally obtained the cleaned dataset with no missing values with all data in its proper format.


<h2 id="data_standardization">Data Standardization</h2>
<p>
Data is usually collected from different agencies in different formats.
(Data standardization is also a term for a particular type of data normalization where we subtract the mean and divide by the standard deviation.)
</p>

<b>What is standardization?</b>

<p>Standardization is the process of transforming data into a common format, allowing the researcher to make the meaningful comparison.
</p>

<b>Example</b>

<p>Transform months to years:</p>
<p>In our dataset, there are two columns in which the values are presented in months.</p>
<p>We will need to apply <b>data transformation</b> to transform months to years.</p>


<p>We can do many mathematical operations directly in Pandas.</p>


In [ ]:
df.head()

In [ ]:
# Convert months to years by mathematical operation (Divide months by 12)
df['Overall Survival (Years)'] = df["Overall Survival (Months)"]/12

df = df.drop(columns='Overall Survival (Months)')

# check your transformed data
df.head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #2: </h1>

<b>According to the example above, transform months to years in the column of "Relapse Free Status (Years)".</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
# Convert months to years by mathematical operation (Divide months by 12)
df["Relapse Free Status (Years)"] = df["Relapse Free Status (Months)"]/12

# check your transformed data
df.head()

```

</details>


<h2 id="data_normalization">Data Normalization</h2>

<b>Why normalization?</b>

<p>Normalization is the process of transforming values of several variables into a similar range. Typical normalizations include scaling the variable so the variable average is 0, scaling the variable so the variance is 1, or scaling the variable so the variable values range from 0 to 1.
</p>

<b>Example</b>

<p>To demonstrate normalization, let's say we want to scale the column "Tumor Size".</p>
<p><b>Target:</b> would like to normalize this variable so their value ranges from 0 to 1</p>


In [ ]:
# replace (original value) by MinMaxScaler values

scaler = preprocessing.MinMaxScaler()
df['Tumor Size']  = scaler.fit_transform(df[['Tumor Size']])

# show the scaled column
df[["Tumor Size"]].head()

<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question #3: </h1>

<b>According to the task above, normalize the data by another method. Replace original value by (original value)/(maximum value)</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
df['Tumor Size'] = df['Tumor Size']/df['Tumor Size'].max()

# show the scaled column
df[["Tumor Size"]].head()


```

</details>


Here we can see we've normalized column "Tumor Size" in the range of \[0,1].


<h2 id="binning">Binning</h2>
<b>Why binning?</b>
<p>
    Binning is a process of transforming continuous numerical variables into discrete categorical 'bins' for grouped analysis.
</p>

<b>Example: </b>

<p>In our dataset, "Nottingham prognostic index" is a real valued variable ranging from 1 to 7. What if we only care about the difference between patients with high index, medium index, and little index (3 types)? Can we rearrange them into three ‘bins' to simplify analysis? </p>

<p>We will use the pandas method 'cut' to segment the 'Nottingham prognostic index' column into 3 bins.</p>


<h3>Example of Binning Data In Pandas</h3>


Convert data to correct format:


In [ ]:
df["Nottingham prognostic index"]=df["Nottingham prognostic index"].astype(int, copy=True)

Let's plot the histogram of Nottingham prognostic index to see what the distribution of Nottingham prognostic index looks like.



In [ ]:
plt.pyplot.hist(df["Nottingham prognostic index"])

# set x/y labels and plot title
plt.pyplot.xlabel("Nottingham prognostic index")
plt.pyplot.ylabel("count")
plt.pyplot.title("Nottingham prognostic index bins")

<p>We would like 3 bins of equal size bandwidth so we use numpy's <code>linspace(start_value, end_value, numbers_generated</code> function.</p>
<p>Since we want to include the minimum value of Nottingham prognostic index, we want to set start_value = min(df["Nottingham prognostic index"]).</p>
<p>Since we want to include the maximum value of Nottingham prognostic index, we want to set end_value = max(df["Nottingham prognostic index"]).</p>
<p>Since we are building 3 bins of equal length, there should be 4 dividers, so numbers_generated = 4.</p>


We build a bin array with a minimum value to a maximum value by using the bandwidth calculated above. The values will determine when one bin ends and another begins.


In [ ]:
bins = np.linspace(min(df["Nottingham prognostic index"]), max(df["Nottingham prognostic index"]), 4)
bins

We set group  names:


In [ ]:
group_names = ['Low', 'Medium', 'High']

We apply the function "cut" to determine what each value of `df['Nottingham prognostic index']` belongs to.


In [ ]:
df['Nottingham prognostic index-binned'] = pd.cut(df['Nottingham prognostic index'], bins, labels=group_names, include_lowest=True )
df[['Nottingham prognostic index','Nottingham prognostic index-binned']].head(20)

Let's see the number of patients in each bin:


In [ ]:
df["Nottingham prognostic index-binned"].value_counts()

Let's plot the distribution of each bin:



In [ ]:
pyplot.bar(group_names, df["Nottingham prognostic index-binned"].value_counts())

# set x/y labels and plot title
plt.pyplot.xlabel("Nottingham prognostic index")
plt.pyplot.ylabel("count")
plt.pyplot.title("Nottingham prognostic index bins")

<p>
    Look at the dataframe above carefully. You will find that the last column provides the bins for "Nottingham prognostic index" based on 3 categories ("Low", "Medium" and "High").
</p>


<h3>Bins Visualization</h3>
Normally, a histogram is used to visualize the distribution of bins we created above. 




In [ ]:
# draw histogram of attribute "Nottingham prognostic index" with bins = 3
plt.pyplot.hist(df["Nottingham prognostic index"], bins = 3)

# set x/y labels and plot title
plt.pyplot.xlabel("Nottingham prognostic index")
plt.pyplot.ylabel("count")
plt.pyplot.title("Nottingham prognostic index bins")

The plot above shows the binning result for the attribute "Nottingham prognostic index".


<h2 id="indicator">Indicator Variable (or Dummy Variable)</h2>
<b>What is an indicator variable?</b>
<p>
    An indicator variable (or dummy variable) is a numerical variable used to label categories. They are called 'dummies' because the numbers themselves don't have inherent meaning. 
</p>

<b>Why we use indicator variables?</b>

<p>
    We use indicator variables so we can use categorical variables for regression analysis in the later modules.
</p>
<b>Example</b>
<p>
    We see the column "Inferred Menopausal State" has two unique values: "Post" or "Pre". Regression doesn't understand words, only numbers. To use this attribute in regression analysis, we convert "Inferred Menopausal State" to indicator variables.
</p>

<p>
    We will use pandas' method 'get_dummies' to assign numerical values to different categories of fuel type. 
</p>


In [ ]:
df.columns

Get the indicator variables and assign it to data frame "dummy_variable\_1":


In [ ]:
dummy_variable_1 = pd.get_dummies(df["Inferred Menopausal State"])
dummy_variable_1.head()

Change the column names for clarity:


In [ ]:
dummy_variable_1.rename(columns={'Post':'Inferred Menopausal State-Post',
                                 'Pre':'Inferred Menopausal State-Pre'}, inplace=True)
dummy_variable_1.head()

In the dataframe, column 'Inferred Menopausal State' has values for 'Post' and 'Pre' as 0s and 1s now.


In [ ]:
# merge data frame "df" and "dummy_variable_1" 
df = pd.concat([df, dummy_variable_1], axis=1)

# drop original column "Inferred Menopausal State" from "df"
df.drop("Inferred Menopausal State", axis = 1, inplace=True)

In [ ]:
df.head()

The last two columns are now the indicator variable representation of the Inferred Menopausal State variable. They're all 0s and 1s now.


<div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #4: </h1>

<b>Similar to before, create an indicator variable for the column "Relapse Free Status"</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute


<details><summary>Click here for the solution</summary>

```python
# get indicator variables of aspiration and assign it to data frame "dummy_variable_2"
dummy_variable_2 = pd.get_dummies(df['Relapse Free Status'])

# change column names for clarity
dummy_variable_2.rename(columns={'Recurred':'Relapse Free Status-Recurred', 'Not Recurred': 'Relapse Free Status-Not Recurred'}, inplace=True)

# show first 5 instances of data frame "dummy_variable_1"
dummy_variable_2.head()
```

</details>


 <div class="alert alert-danger alertdanger" style="margin-top: 20px">
<h1> Question  #5: </h1>

<b>Merge the new dataframe to the original dataframe, then drop the column 'aspiration'.</b>

</div>


In [ ]:
# Write your code below and press Shift+Enter to execute 


<details><summary>Click here for the solution</summary>

```python
# merge the new dataframe to the original datafram
df = pd.concat([df, dummy_variable_2], axis=1)

# drop original column "aspiration" from "df"
df.drop('Relapse Free Status', axis = 1, inplace=True)
```

</details>


In [ ]:
df.to_csv('breast_cancer_clean')

Save the new csv:

> Note : The  csv file cannot be viewed in the jupyterlite based SN labs environment.However you can Click <a href="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Module%202/DA0101EN-2-Review-Data-Wrangling.ipynb?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkDA0101ENSkillsNetwork20235326-2022-01-01">HERE</a> to download the lab notebook (.ipynb) to your local machine and view the csv file once the notebook is executed.


### Thank you for completing this lab!

## Author

<a href="https://author.skills.network/instructors/dmytro_shliakhovskyi">Dmytro Shliakhovskyi</a>

### Other Contributors

<a href="https://author.skills.network/instructors/yaroslav_vyklyuk_2">Prof. Yaroslav Vyklyuk, DrSc, PhD</a>

<a href="https://author.skills.network/instructors/nataliya_boyko">Ass. Prof. Nataliya Boyko, PhD</a>


## Change Log

| Date (YYYY-MM-DD) | Version | Changed By | Change Description                                         |
| ----------------- | ------- | ---------- | ---------------------------------------------------------- |
|    2023-03-04     | 01 | Dmytro Shliakhovkyi | Lab created |



<hr>

## <h3 align="center"> © IBM Corporation 2020. All rights reserved. <h3/>
